In [1]:
%pwd

'c:\\Users\\lenov\\OneDrive\\Documents\\wine_quality\\notebooks'

In [2]:
import os
os.chdir('..')
%pwd

'c:\\Users\\lenov\\OneDrive\\Documents\\wine_quality'

In [2]:
from src.config.common_utils import read_yaml, create_directories

In [3]:
from src.config.config import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [5]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            self.schema = read_yaml(schema_filepath)
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
          config = self.config.data_ingestion

          create_directories([config.root_dir])

          data_ingestion_config = DataIngestionConfig(
                root_dir= config.root_dir,
                source_url=config.source_URL,
                local_data_file=config.local_data_file,
                unzip_dir=config.unzip_dir
          )

          return data_ingestion_config

In [11]:
import urllib.request as request
import os
from src.config.logger import logger
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! With following info: \n{headers}")
        
        else:
            logger.info(f"File already exists.")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [13]:
config = ConfigurationManager()
data_ingestion_config = config.get_data_ingestion_config()
data_ingestion = DataIngestion(config = data_ingestion_config)
data_ingestion.download_file()
data_ingestion.extract_zip_file()

[2024-06-25 16:15:41,501: INFO: common_utils: yaml file: src\config\config.yaml loaded successfully]
[2024-06-25 16:15:41,502: INFO: common_utils: yaml file: src\models\params.yaml loaded successfully]
[2024-06-25 16:15:41,508: INFO: common_utils: yaml file: src\data\schema.yaml loaded successfully]
[2024-06-25 16:15:41,511: INFO: common_utils: created directory at: data/raw]
[2024-06-25 16:15:42,562: INFO: 750345593: data/raw/data.zip download! With following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 134B:2C0FE0:63C316:68CCE8:667AD0BA
Accept-Ranges: bytes
Date: Tue, 25 Jun 2024 14:15:41 GMT
Via: 1.1 varnish
X-Served-By: cache

NameError: name 'zipfile' is not defined